In [1]:
import os
os.getcwd( )

'C:\\Users\\Juan\\nlp_intro'

In [2]:
os.chdir('D:\\Jupyter projects\\nlp_intro') 

In [3]:
pwd

'D:\\Jupyter projects\\nlp_intro'

In [4]:
# Check for GPU
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3060 (UUID: GPU-e8861a5c-89dd-5ee2-06b2-d3bca4ee7bc6)


In [5]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-10-31 12:20:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: 'helper_functions.py.2'

     0K ..........                                            100% 2.95M=0.003s

2022-10-31 12:20:40 (2.95 MB/s) - 'helper_functions.py.2' saved [10246/10246]



In [6]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [7]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-10-31 12:20:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.212.240, 142.250.179.240, 142.250.180.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.212.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip.3'

     0K .......... .......... .......... .......... ..........  8% 2.07M 0s
    50K .......... .......... .......... .......... .......... 16% 3.60M 0s
   100K .......... .......... .......... .......... .......... 25% 7.24M 0s
   150K .......... .......... .......... .......... .......... 33% 9.22M 0s
   200K .......... .......... .......... .......... .......... 42% 16.4M 0s
   250K .......... .......... .......... .......... .......... 50% 1.96M 0s
   300K .......... .......... .......... .......... .......... 59% 56.0M 0s
   350K .......... .......... .......

In [8]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
# The test data doesn't have a target (that's what we'd try to predict)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [13]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
Newberg upheaval: Jacque Betz responds 'looking forward to the day' she can answerÛ_ http://t.co/LzasR05ljo  #news http://t.co/IeMxGSE2BE

---

Target: 0 (not real disaster)
Text:
cancel the fucking show. Evacuate MetLife  https://t.co/SkQ8oUcM3R

---

Target: 1 (real disaster)
Text:
Sooo police dispatch said there was a person threatening to shoot up the Walmart on Rutherford &amp; they had to evacuate

---

Target: 0 (not real disaster)
Text:
L B #Entertainment lot of 8 #BruceWillis MOVIES #DVD DIE HARD 1 2 12 MONKEYS ARMAGEDDON SIXTH #eBay #Auction http://t.co/CxDJApzXMP

---

Target: 1 (real disaster)
Text:
A sandstorm in Jordan has coated the Middle EastÛªs largest refugee camp in a layer of grit http://t.co/hVJmuuaLXV http://t.co/T8Nz6h9Zz4

---



In [14]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [15]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [17]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None
)

In [18]:
#find average number of tokens/words in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length
)

In [20]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [21]:
#create sample sentence
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [22]:
#choose random
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
'Redskins WR Roberts Belly-Bombed ' via @TeamStream http://t.co/GbcvVEvDTY      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[9261, 6668, 3461,    1,   49, 4457,    1,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [23]:
#Get unique words in vocab
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] #5 most common tokens
bottom_5_words = words_in_vocab[-5:]# 5 least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [24]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length,
                             name='embedding_1'
                            )
embedding

In [25]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

#embed random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
@crabbycale OH MY GOD THE MEMORIES ARE FLOODING BACK      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04419341, -0.02349438,  0.00897091, ..., -0.02296398,
          0.03352927, -0.03165143],
        [-0.00158058,  0.04919602, -0.03033113, ..., -0.02477709,
          0.03940349,  0.03351363],
        [ 0.01360643,  0.01858305,  0.01822582, ..., -0.02490723,
         -0.0353555 , -0.0194996 ],
        ...,
        [-0.03226759, -0.00171774,  0.0351618 , ...,  0.02675134,
          0.01835357,  0.03775084],
        [-0.03226759, -0.00171774,  0.0351618 , ...,  0.02675134,
          0.01835357,  0.03775084],
        [-0.03226759, -0.00171774,  0.0351618 , ...,  0.02675134,
          0.01835357,  0.03775084]]], dtype=float32)>

In [26]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.04419341, -0.02349438,  0.00897091, -0.04458587, -0.01814613,
        0.00300959, -0.02228291, -0.00437361,  0.02259595, -0.00011273,
       -0.04051753,  0.00835348, -0.04182898,  0.0056477 ,  0.01021483,
       -0.00122981, -0.02769551,  0.03072769, -0.00697983,  0.0458906 ,
        0.01045223,  0.00588731,  0.00919248,  0.00808529, -0.0021399 ,
        0.02289527, -0.01313497, -0.01990033, -0.03011679, -0.00606819,
        0.04393696, -0.03031437, -0.02962606,  0.01701239, -0.01730274,
        0.04995744, -0.04426212, -0.03533043, -0.00429139,  0.02145872,
       -0.04886849, -0.03404929, -0.03237206,  0.02334782,  0.01943706,
        0.03244599, -0.00215797,  0.03356018, -0.02156054, -0.02636491,
        0.00458449, -0.01652144,  0.02260948, -0.01586981, -0.03174639,
       -0.03823803, -0.03235634, -0.00475132,  0.04063277, -0.04728155,
       -0.00602496,  0.02929956,  0.02735356, -0.04892446, -0.04926798,
       -0.049772

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokenization and modelling pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [28]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [29]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [30]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  
    Args:
    -----
    y_true = true labels in the form of a 1D array
    y_pred = predicted labels in the form of a 1D array
  
    Returns a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    
    # Calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                    "precision": model_precision,
                    "recall": model_recall,
                    "f1": model_f1}
    
    return model_results

In [31]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [32]:
#MODEL 1

from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [33]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype='string') # inputs are 1D strings
x = text_vectorizer(inputs) # text input into numbers
x = embedding(x) # create embedding of numerized numbers
x = layers.GlobalAveragePooling1D()(x) # 

outputs = layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [34]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy']
)

In [35]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [36]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name="simple_dense_model")]
)

Saving TensorBoard log files to: model_logs/simple_dense_model/20221031-143555
Epoch 1/5
215/215 [==============================] - 6s 7ms/step - loss: 0.6098 - accuracy: 0.6945 - val_loss: 0.5365 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4415 - accuracy: 0.8187 - val_loss: 0.4695 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3467 - accuracy: 0.8603 - val_loss: 0.4593 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2852 - accuracy: 0.8918 - val_loss: 0.4644 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2383 - accuracy: 0.9124 - val_loss: 0.4769 - val_accuracy: 0.7874


In [37]:
# Check the results
model_1.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 4ms/step - loss: 0.4769 - accuracy: 0.7874


[0.47693932056427, 0.787401556968689]

In [38]:
embedding.weights


[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.04223466, -0.01232036,  0.02492392, ...,  0.03739414,
          0.02754631,  0.0269674 ],
        [ 0.03788092, -0.02968534,  0.00245902, ..., -0.01668192,
          0.03793933, -0.03829484],
        [-0.04201521, -0.03491365, -0.00389276, ...,  0.06751584,
          0.07394353, -0.04339752],
        ...,
        [ 0.01947466,  0.01368252, -0.01293451, ...,  0.04440505,
          0.04755535,  0.04615953],
        [-0.06935913, -0.0707734 , -0.07123754, ...,  0.03798301,
          0.05890796, -0.00278805],
        [-0.11110515, -0.02027354, -0.11476819, ...,  0.08226258,
          0.0864292 , -0.04234757]], dtype=float32)>]

In [39]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [40]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.4020229 ],
       [0.7458195 ],
       [0.99794346],
       [0.10876183],
       [0.11108602],
       [0.94066375],
       [0.9065157 ],
       [0.9925392 ],
       [0.96853405],
       [0.26828274]], dtype=float32)

In [41]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

In [43]:
# Is our simple Keras model better than our baseline model?
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [44]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
    for key, value in baseline_results.items():
        print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results, 
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 78.74, Difference: -0.52
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.01
Baseline f1: 0.79, New f1: 0.78, Difference: -0.00


In [45]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [46]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [47]:
# Get the weight matrix of embedding layer 
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)

(10000, 128)


In [49]:
tf.random.set_seed(42)

from tensorflow.keras import layers

model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer='uniform',
                                     input_length=max_length,
                                     name="embedding_2"
                                    )

inputs = layers.Input(shape=(1, ), dtype='string')
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs=layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [50]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [51]:
model_2.summary()


Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [52]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20221031-172408
Epoch 1/5
215/215 [==============================] - 5s 10ms/step - loss: 0.5136 - accuracy: 0.7393 - val_loss: 0.4599 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3205 - accuracy: 0.8704 - val_loss: 0.5084 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2211 - accuracy: 0.9156 - val_loss: 0.5811 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1572 - accuracy: 0.9444 - val_loss: 0.6095 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1117 - accuracy: 0.9583 - val_loss: 0.7912 - val_accuracy: 0.7572


In [53]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

24/24 [==============================] - 0s 6ms/step


((762, 1),
 array([[0.02688102],
        [0.71889037],
        [0.9995449 ],
        [0.0646049 ],
        [0.00470248],
        [0.999308  ],
        [0.85061496],
        [0.9997037 ],
        [0.9995945 ],
        [0.20401447]], dtype=float32))

In [54]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [55]:

# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 75.7217847769029,
 'precision': 0.7579616566971975,
 'recall': 0.7572178477690289,
 'f1': 0.7553466343855008}

In [56]:
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 79.27, New accuracy: 75.72, Difference: -3.54
Baseline precision: 0.81, New precision: 0.76, Difference: -0.05
Baseline recall: 0.79, New recall: 0.76, Difference: -0.04
Baseline f1: 0.79, New f1: 0.76, Difference: -0.03


In [57]:
tf.random.set_seed(42)

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer='uniform',
                                     input_length=max_length,
                                     name='embedding_3'
                                     )

inputs = layers.Input(shape=(1, ), dtype='string')
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [59]:
model_3.compile(loss="binary_crossentropy",
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=['accuracy']
                         )

In [60]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [61]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5, 
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")]
)

Saving TensorBoard log files to: model_logs/GRU/20221031-175908
Epoch 1/5
215/215 [==============================] - 3s 9ms/step - loss: 0.5299 - accuracy: 0.7221 - val_loss: 0.4535 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3198 - accuracy: 0.8680 - val_loss: 0.4888 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2189 - accuracy: 0.9177 - val_loss: 0.5626 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1562 - accuracy: 0.9463 - val_loss: 0.6227 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1194 - accuracy: 0.9600 - val_loss: 0.5949 - val_accuracy: 0.7717


In [63]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 0s 5ms/step


((762, 1),
 array([[0.3066615 ],
        [0.90319353],
        [0.9966588 ],
        [0.14305897],
        [0.01514528],
        [0.9873284 ],
        [0.6025643 ],
        [0.9967751 ],
        [0.9953602 ],
        [0.37042946]], dtype=float32))

In [64]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [65]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.7715668446582516,
 'recall': 0.7716535433070866,
 'f1': 0.7706989403422689}

In [66]:
# Compare to baseline
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy: 79.27, New accuracy: 77.17, Difference: -2.10
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.02
Baseline f1: 0.79, New f1: 0.77, Difference: -0.02


In [68]:
tf.random.set_seed(42)

model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                    output_dim=128,
                                    embeddings_initializer='uniform',
                                    input_length=max_length,
                                    name='embedding_4')

inputs = layers.Input(shape=(1, ), dtype='string')
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_Bidirectional')

In [69]:
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy']
               )

In [70]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [71]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20221031-181524
Epoch 1/5
215/215 [==============================] - 5s 13ms/step - loss: 0.5128 - accuracy: 0.7414 - val_loss: 0.4594 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3154 - accuracy: 0.8718 - val_loss: 0.5155 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2136 - accuracy: 0.9194 - val_loss: 0.5762 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1454 - accuracy: 0.9518 - val_loss: 0.6493 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1027 - accuracy: 0.9647 - val_loss: 0.6973 - val_accuracy: 0.7664


In [72]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.04355242],
       [0.8433966 ],
       [0.9995875 ],
       [0.10725769],
       [0.00377868],
       [0.99506384],
       [0.97462577],
       [0.99960226],
       [0.99953735],
       [0.19197886]], dtype=float32)

In [73]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [74]:
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 76.64041994750657,
 'precision': 0.7664644290702826,
 'recall': 0.7664041994750657,
 'f1': 0.765226875463096}

In [75]:
compare_baseline_to_new_results(baseline_results, model_4_results)

Baseline accuracy: 79.27, New accuracy: 76.64, Difference: -2.62
Baseline precision: 0.81, New precision: 0.77, Difference: -0.04
Baseline recall: 0.79, New recall: 0.77, Difference: -0.03
Baseline f1: 0.79, New f1: 0.77, Difference: -0.02


In [76]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=32, kernel_size=5, activation='relu')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [77]:
# See the outputs of each layer
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.03528265, -0.02572645, -0.00296372, ..., -0.00724024,
          -0.00215557,  0.01195835],
         [-0.07590447,  0.00535872,  0.00499065, ..., -0.01406056,
           0.04207451, -0.05433482],
         [-0.04285926, -0.00275673,  0.03422695, ..., -0.03037849,
           0.03506533, -0.01866705],
         ...,
         [-0.04223466, -0.01232036,  0.02492392, ...,  0.03739414,
           0.02754631,  0.0269674 ],
         [-0.04223466, -0.01232036,  0.02492392, ...,  0.03739414,
           0.02754631,  0.0269674 ],
         [-0.04223466, -0.01232036,  0.02492392, ...,  0.03739414,
           0.02754631,  0.0269674 ]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.        , 0.01005853, 0.        , 0.        , 0.        ,
          0.        , 0.02544659, 0.        , 0.08993326, 0.01290459,
          0.049881  , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.

In [78]:
tf.random.set_seed(42)

model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer='uniform',
                                     input_length=max_length,
                                     name='embedding_5')

inputs = layers.Input(shape=(1, ), dtype='string')
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name='model_5_Conv1D')

model_5.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

model_5.summary()


Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [79]:
model_5_history = model_5.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences, val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR, 'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20221031-194318
Epoch 1/5
215/215 [==============================] - 4s 7ms/step - loss: 0.5729 - accuracy: 0.7069 - val_loss: 0.4738 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3439 - accuracy: 0.8580 - val_loss: 0.4682 - val_accuracy: 0.7979
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2107 - accuracy: 0.9219 - val_loss: 0.5301 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1344 - accuracy: 0.9553 - val_loss: 0.6019 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0956 - accuracy: 0.9682 - val_loss: 0.6570 - val_accuracy: 0.7913


In [82]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5]

24/24 [==============================] - 0s 3ms/step


array([[0.4409066 ],
       [0.645164  ],
       [0.9998053 ],
       [0.08580104],
       [0.001558  ]], dtype=float32)

In [83]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [84]:
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)

model_5_results

{'accuracy': 79.13385826771653,
 'precision': 0.7939542015160368,
 'recall': 0.7913385826771654,
 'f1': 0.7892663801484977}

In [85]:
compare_baseline_to_new_results(baseline_results, model_5_results)


Baseline accuracy: 79.27, New accuracy: 79.13, Difference: -0.13
Baseline precision: 0.81, New precision: 0.79, Difference: -0.02
Baseline recall: 0.79, New recall: 0.79, Difference: -0.00
Baseline f1: 0.79, New f1: 0.79, Difference: 0.00


In [87]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

UnknownError: Graph execution error:

JIT compilation failed.
	 [[{{node EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/embedding_lookup/mod}}]] [Op:__inference_restored_function_body_57069]

In [ ]:
# Each sentence has been encoded into a 512 dimension vector
embed_samples[0].shape

In [88]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name='USE'
                                       )

In [90]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
], name='model_6_USE')

model_6.compile(loss='binary_crossentropy',
                optimizer='Adam',
                metrics=['accuracy']
               )

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [91]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20221031-201237
Epoch 1/5


UnknownError: Graph execution error:

JIT compilation failed.
	 [[{{node EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/embedding_lookup/mod}}]] [Op:__inference_train_function_67092]